In [ ]:
#import libraries used in this file
import geo_tools as gt
import pandas as pd 
import numpy as np

In [ ]:
#load in the arthritis dataset
data = pd.read_pickle('Dataset/arth raw.pkl')

In [ ]:
#Transpose our data such that each row is a GSM. 
data_test = data.T
 
data_test.columns.name = "CpG"

In [ ]:
#Load in the ages of our arthritis dataset
ages = pd.read_pickle('Dataset/arth ages.pkl')
ages = ages.rename(columns={ages.columns[0]: 'Age'})

In [ ]:
#Load in the disease status of each GSM
arth_status = pd.read_pickle('Dataset/arth status.pkl')
arth_status = arth_status.rename(columns={arth_status.columns[0]: 'status'})

In [ ]:
#Select those GSMs with disease status of rheumatoid arthritis 
arth = arth_status[arth_status["status"] == "rheumatoid arthritis"]

#Select those GSMs with disease status of normal 
norm = arth_status[arth_status["status"] == "normal"]

In [ ]:
#Generate a table of Beta values for each CpG for each GSM in the "normal" subgroup 
common_cols = arth_status.index.intersection(norm.index).to_list()
norm_test = data_test.loc[common_cols]

In [ ]:
#Generate a table of age for each GSM in the "normal" subgroup 
norm_ages = ages.loc[common_cols]

In [ ]:
#Generate a table of Beta values for each CpG for each GSM in the "arthritis" subgroup 
common_cols = arth_status.index.intersection(arth.index).to_list()
arth_test = data_test.loc[common_cols]

In [ ]:
#Generate a table of age for each GSM in the "normal" subgroup 
arth_ages = ages.loc[common_cols]

In [ ]:
#Load the R^2 value of each CpG in the healthy subset 
Norm_Stats = pd.read_pickle('CpG_Rs/CpG_R_healthy_arth.pkl')

In [ ]:
#rank the CpGs by Rvalue, then given a rank
Norm_Stats = Norm_Stats.reindex(Norm_Stats.Rvalue.abs().sort_values(ascending=False, kind='mergesort').index)
Norm_Stats["Rank"] = [i for i in range(1,len(Norm_Stats.index)+1)]

In [ ]:
#ensure that the R value is less that 0.05 and
Norm_Stats_low = Norm_Stats[(Norm_Stats["Rvalue"]<=0.0025)]

In [ ]:
# Load R^2 values of CpGs across the entire dataset
dis_Stats = pd.read_pickle('CpG_Rs/CpG_R_arth_dis.pkl')

In [ ]:
# reorganize this table from highest to lowest absolute rvalue and then rank CpGs from most to least correlated 
dis_Stats = dis_Stats.reindex(dis_Stats.Rvalue.abs().sort_values(ascending=False, kind='mergesort').index) 
dis_Stats["Rank"] = [i for i in range(1,len(dis_Stats.index)+1)]

In [ ]:
#ensure that the Rvalue is less that 0.05 and
dis_Stats_low = dis_Stats[(dis_Stats["Rvalue"]<=0.0025)]

In [ ]:
#generate the HIV+ male dataset
HIV_data = gt.matrix_to_df("data_GSE/GSE53840/GSE53840_series_matrix.txt", GSE = "GSE53840")

In [ ]:
#Transpose our data such that each row is a GSM. HIV_test = HIV_data.T
HIV_test.columns.name = "CpG"

In [ ]:
# find the age of each GSM HiV_age = []
for item in HIV_data.columns.to_list(): HiV_age.append(gt.info("GSE53840", item))
# Generate a list of each GSM's age
HiV_ages = pd.DataFrame(data=HiV_age, index=HIV_data.columns.to_list()) 
HiV_ages = HiV_ages.rename(columns={HiV_ages.columns[0]: 'Age'})

In [ ]:
HIV_Stats = pd.read_pickle('CpG_Rs/CpG_R_HIV.pkl')

In [ ]:
# reorganize this table from highest to lowest absolute rvalue and then rank CpGs from most to least correlated
 
HIV_Stats = HIV_Stats.reindex(HIV_Stats.Rvalue.abs().sort_values(ascending=False, kind='mergesort').index) 
HIV_Stats["Rank"] = [i for i in range(1,len(HIV_Stats.index)+1)]

In [ ]:
#ensure that the Rvalue is less that 0.05 and
HIV_Stats_low = HIV_Stats[(HIV_Stats["Rvalue"]<=0.0025)]

In [ ]:
#Find CpGs with R <= 0.05 in each considered dataset
common_low = set(Norm_Stats_low["CpG"]).intersection(dis_Stats_low["CpG"]) 
common_low = pd.Series(list(common_low.intersection(HIV_Stats_low["CpG"]))).to_list()

In [ ]:
lowestR_Betas_n = norm_test[norm_test.columns.intersection(common_low)] 
lowestR_Betas_n["Age"] = norm_ages

lowestR_Betas_a = arth_test[arth_test.columns.intersection(common_low)] 
lowestR_Betas_a["Age"] = arth_ages

lowestR_Betas_h = HIV_test[HIV_test.columns.intersection(common_low)] 
lowestR_Betas_h["Age"] = HiV_ages

In [ ]:
#Generate Bins with 5 year interval from 25 to 90. Then Bins are created for all samples bellow 25 and above 90. 
# Then place GSMs into Bin according to Age. The largest bin is given the uper age limit of 101.
i = 25

age_Betas = [] 
Age_upper = [] 
temp = 0
Leftover_Beta = lowestR_Betas_h 

while i <= 65:
    temp = Leftover_Beta[(Leftover_Beta["Age"]<= i)]
    Leftover_Beta = Leftover_Beta.drop(temp.index.to_list() , axis = 0) 
    Age_upper.append(i)
    i += 5
    age_Betas.append(temp) 
age_Betas.append(Leftover_Beta) 
Age_upper.append(70)

In [ ]:
#find standard deviations stds = []
for age in range(len(age_Betas)): 
    stds.append(age_Betas[age].T[:-1].T.std())
stds_df_h = pd.DataFrame( stds, index = Age_upper, columns = lowestR_Betas_h.columns.to_list()[:-1])

In [ ]:
#Generate Bins with 5 year interval from 25 to 90. Then Bins are created for all samples bellow 25 and above 90. # Then place GSMs into Bin according to Age. The largest bin is given the uper age limit of 101.
i = 25
age_Betas = [] 
Age_upper = [] 
temp = 0
Leftover_Beta = lowestR_Betas_n 
while i <= 65:
    temp = Leftover_Beta[(Leftover_Beta["Age"]<= i)]
    Leftover_Beta = Leftover_Beta.drop(temp.index.to_list() , axis = 0) Age_upper.append(i)
    i += 5
    age_Betas.append(temp) 
age_Betas.append(Leftover_Beta) 
Age_upper.append(70)

In [ ]:
#find standard deviations stds = []
for age in range(len(age_Betas)): 
    stds.append(age_Betas[age].T[:-1].T.std())
stds_df_n = pd.DataFrame( stds, index = Age_upper, columns = lowestR_Betas_n.columns.to_list()[:-1])

In [ ]:
#Generate Bins with 5 year interval from 25 to 90. Then Bins are created for all samples bellow 25 and above 90. # Then place GSMs into Bin according to Age. The largest bin is given the uper age limit of 101.
i = 25
age_Betas = [] 
Age_upper = [] 
temp = 0
Leftover_Beta = lowestR_Betas_a 

while i <= 65:
    temp = Leftover_Beta[(Leftover_Beta["Age"]<= i)]
    Leftover_Beta = Leftover_Beta.drop(temp.index.to_list() , axis = 0) 
    Age_upper.append(i)
    i += 5
    age_Betas.append(temp) 
age_Betas.append(Leftover_Beta) 
Age_upper.append(70)

In [ ]:
#find standard deviations stds = []
for age in range(len(age_Betas)): 
    stds.append(age_Betas[age].T[:-1].T.std())
stds_df_a = pd.DataFrame( stds, index = Age_upper, columns = lowestR_Betas_a.columns.to_list()[:-1])